In [1]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# E2E ML on GCP: MLOps stage 2 : AutoML Image Classfication Training with Customer Managed Encryption Keys (CMEK)
<table align="left">
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/ml_ops/stage2/get_started_with_cmek_training.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/ai/platform/notebooks/deploy-notebook?download_url=https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/ml_ops/stage2/get_started_with_cmek_training.ipynb">
      Open in Google Cloud Notebooks
    </a>
  </td>
</table>
<br/><br/><br/>

## Overview


This tutorial demonstrates how to use Vertex AI for E2E MLOps on Google Cloud in production. This tutorial covers stage 2 : experimentation: get started with AutoML training with a customer managed encyrption key CMEK.

### Dataset

The dataset used for this tutorial is the [Flowers dataset](https://www.tensorflow.org/datasets/catalog/tf_flowers) from [TensorFlow](https://www.tensorflow.org/datasets/catalog/overview). The version of the dataset you will use in this tutorial is stored in a public #(GCS) bucket. The trained model predicts the type of flower an image is from a class of five flowers: daisy, dandelion, rose, sunflower, or tulip.


### Objective

In this tutorial, you learn how to use a customer managed encryption key (CMEK) for `Vertex AI AutoML` training.

This tutorial uses the following Google Cloud ML services:

- `Vertex AI AutoML`
- Customer managed encryption key.

The steps performed include:

- Creating a customer managed encryption key.
- Creating an image dataset with CMEK encryption.
- Train an AutoML model with CMEK encryption.

## Installations

Install the Vertex AI SDK and the KMS package for CMEK encryption.

In [2]:
USER_FLAG="--user"

! pip3 install --upgrade google-cloud-aiplatform $USER_FLAG
! pip3 install --upgrade google-cloud-kms $USER_FLAG

  Using cached google_cloud_aiplatform-1.11.0-py2.py3-none-any.whl (1.8 MB)
  Using cached grpcio-1.44.0-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (4.3 MB)
  Attempting uninstall: google-cloud-aiplatform
    Found existing installation: google-cloud-aiplatform 1.10.0
    Uninstalling google-cloud-aiplatform-1.10.0:
      Successfully uninstalled google-cloud-aiplatform-1.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tfx 1.2.0 requires absl-py<0.13,>=0.9, but you have absl-py 0.15.0 which is incompatible.
tfx 1.2.0 requires attrs<21,>=19.3.0, but you have attrs 21.4.0 which is incompatible.
tfx 1.2.0 requires click<8,>=7, but you have click 8.0.4 which is incompatible.
tfx 1.2.0 requires docker<5,>=4.1, but you have docker 5.0.3 which is incompatible.
tfx 1.2.0 requires google-cloud-aiplatform<0.8,>=0.5.0, but you have google-cloud-aiplatf

### Restart the kernel

Once you've installed the additional packages, you need to restart the notebook kernel so it can find the packages.

In [3]:
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython
    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

#### Set your project ID

**If you don't know your project ID**, you may be able to get your project ID using `gcloud`.

In [4]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [5]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = ! gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

Project ID: andy-1234-221921


In [6]:
! gcloud config set project $PROJECT_ID

Updated property [core/project].


#### Region

You can also change the `REGION` variable, which is used for operations
throughout the rest of this notebook.  Below are regions supported for Vertex AI. We recommend that you choose the region closest to you.

- Americas: `us-central1`
- Europe: `europe-west4`
- Asia Pacific: `asia-east1`

You may not use a multi-regional bucket for training with Vertex AI. Not all regions provide support for all Vertex AI services.

Learn more about [Vertex AI regions](https://cloud.google.com/vertex-ai/docs/general/locations).

In [7]:
REGION = 'us-central1'  # @param {type: "string"}

#### Timestamp

If you are in a live tutorial session, you might be using a shared test account or project. To avoid name collisions between users on resources created, you create a timestamp for each instance session, and append the timestamp onto the name of resources you create in this tutorial.

In [8]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

### Create a Cloud Storage bucket

**The following steps are required, regardless of your notebook environment.**

When you initialize the Vertex SDK for Python, you specify a Cloud Storage staging bucket. The staging bucket is where all the data associated with your dataset and model resources are retained across sessions.

Set the name of your Cloud Storage bucket below. Bucket names must be globally unique across all Google Cloud projects, including those outside of your organization.

In [9]:
BUCKET_NAME = "gs://[your-bucket-name]"  # @param {type:"string"}

In [10]:
if BUCKET_NAME == "" or BUCKET_NAME is None or BUCKET_NAME == "gs://[your-bucket-name]":
    BUCKET_NAME = "gs://" + PROJECT_ID + "aip-" + TIMESTAMP

**Only if your bucket doesn't already exist**: Run the following cell to create your Cloud Storage bucket.

In [11]:
! gsutil mb -l $REGION $BUCKET_NAME

Creating gs://andy-1234-221921aip-20220314170212/...


Finally, validate access to your Cloud Storage bucket by examining its contents:

In [12]:
! gsutil ls -al $BUCKET_NAME

### Set up variables

Next, set up some variables used throughout the tutorial.
### Import libraries and define constants

In [13]:
import google.cloud.aiplatform as aip

from google.cloud import kms

### Initialize Vertex AI SDK for Python

Initialize the Vertex AI SDK for Python for your project and corresponding bucket.

In [14]:
aip.init(project=PROJECT_ID, location=REGION, staging_bucket=BUCKET_NAME)

## Setting up Customer Managed Encryption Keys

By default, Google Cloud automatically encrypts data when it is stored in Cloud Storage using encryption keys managed by Google. If you have specific compliance or regulatory requirements related to the keys that protect your data, you can use customer-managed encryption keys (CMEK) for your training jobs.

### Enable KMS API

First, you enble the [Cloud Key Management Service (KMS)](https://console.cloud.google.com/flows/enableapi?apiid=cloudkms.googleapis.com)

Learn more about [Customer managed encryption keys (CMEK)](https://cloud.google.com/vertex-ai/docs/general/cmek)

### Create a key ring

After you have enabled the KMS API, you create a key ring and a key. Use the helper function `create_key_ring()` to create a key ring, with the following parameters:

- `project_id`: Your project ID.
- `location`: Your region.
- `key_ring_id`: The unique identifier for your key ring.

The helper function calls the KMS client method `create_key_ring()` to create your key ring.

Learn more about [KMS: Create a key ring](https://cloud.google.com/kms/docs/samples/kms-create-key-ring)

In [16]:
KEY_RING_ID = "your_cmek_key_ring_id"


def create_key_ring(project_id, location, key_ring_id):
    """
    Creates a new key ring in Cloud KMS

    Args:
        project_id (string): Google Cloud project ID (e.g. 'my-project').
        location (string): Cloud KMS location (e.g. 'us-east1').
        id (string): ID of the key ring to create (e.g. 'my-key-ring').

    Returns:
        KeyRing: Cloud KMS key ring.

    """

    # Create the client.
    client = kms.KeyManagementServiceClient()

    # Build the parent location name.
    location_name = f"projects/{project_id}/locations/{location}"

    # Build the key ring.
    key_ring = {}

    # Call the API.
    created_key_ring = client.create_key_ring(
        request={"parent": location_name, "key_ring_id": key_ring_id, "key_ring": key_ring}
    )
    print("Created key ring: {}".format(created_key_ring.name))
    return created_key_ring


key_ring = create_key_ring(project_id=PROJECT_ID, location=REGION, key_ring_id=KEY_RING_ID)
print(key_ring)

Created key ring: projects/andy-1234-221921/locations/us-central1/keyRings/your_cmek_key_ring_id


### Create a key

Next, you create your key. Use the helper function `create_key()` with the following parameters:

- `project_id`: Your project ID.
- `location`: Your region.
- `key_ring_id`: The unique identifier for your key ring.
- `key_id`: The unique identifier for your key.

The helper function calls the KMS client method `create_cryto_key()` to create your key.

Learn more about [](https://cloud.google.com/kms/docs/samples/kms-create-key-symmetric-encrypt-decrypt)

In [21]:
KEY_ID = "your_cmek_key_id"

def create_key(project_id, location, key_ring_id, key_id):
    """
    Creates a new symmetric encryption/decryption key in Cloud KMS.

    Args:
        project_id (string): Google Cloud project ID (e.g. 'my-project').
        location (string): Cloud KMS location (e.g. 'us-east1').
        key_ring_id (string): ID of the Cloud KMS key ring (e.g. 'my-key-ring').
        key_id (string): ID of the key to create (e.g. 'my-symmetric-key').

    Returns:
        CryptoKey: Cloud KMS key.

    """

    # Create the client.
    client = kms.KeyManagementServiceClient()

    # Build the parent key ring name.
    key_ring_name = client.key_ring_path(project_id, location, key_ring_id)

    # Build the key.
    purpose = kms.CryptoKey.CryptoKeyPurpose.ENCRYPT_DECRYPT
    algorithm = (
        kms.CryptoKeyVersion.CryptoKeyVersionAlgorithm.GOOGLE_SYMMETRIC_ENCRYPTION
    )
    key = {
        "purpose": purpose,
        "version_template": {
            "algorithm": algorithm,
        },
    }

    # Call the API.
    created_key = client.create_crypto_key(
        request={"parent": key_ring_name, "crypto_key_id": key_id, "crypto_key": key}
    )
    print("Created symmetric key: {}".format(created_key.name))
    return created_key


key_id = create_key(project_id=PROJECT_ID, location=REGION, key_ring_id=KEY_RING_ID, key_id=KEY_ID)

print(key_id)

Created symmetric key: projects/andy-1234-221921/locations/us-central1/keyRings/your_cmek_key_ring_id/cryptoKeys/your_cmek_key_id
name: "projects/andy-1234-221921/locations/us-central1/keyRings/your_cmek_key_ring_id/cryptoKeys/your_cmek_key_id"
primary {
  name: "projects/andy-1234-221921/locations/us-central1/keyRings/your_cmek_key_ring_id/cryptoKeys/your_cmek_key_id/cryptoKeyVersions/1"
  state: ENABLED
  create_time {
    seconds: 1647278023
    nanos: 319485200
  }
  protection_level: SOFTWARE
  algorithm: GOOGLE_SYMMETRIC_ENCRYPTION
  generate_time {
    seconds: 1647278023
    nanos: 319485200
  }
}
purpose: ENCRYPT_DECRYPT
create_time {
  seconds: 1647278023
  nanos: 319485200
}
version_template {
  protection_level: SOFTWARE
  algorithm: GOOGLE_SYMMETRIC_ENCRYPTION
}
destroy_scheduled_duration {
  seconds: 86400
}



### Set service account permissions

Next, you set permissions for your Vertex AI service account to encrypt and decrypt resources using your key.

Learn more about [Grant Vertex AI permissions](https://cloud.google.com/vertex-ai/docs/general/cmek#grant_permissions)

In [23]:
# Reference: https://cloud.google.com/vertex-ai/docs/general/cmek#granting_permissions
# Get the service account
SERVICE_ACCOUNT = ! gcloud projects get-iam-policy {PROJECT_ID} \
  --flatten="bindings[].members" \
  --format="table(bindings.members)" \
  --filter="bindings.role:roles/aiplatform.serviceAgent" \
  | grep -oP "service-.+?@gcp-sa-aiplatform.iam.gserviceaccount.com"
SERVICE_ACCOUNT = SERVICE_ACCOUNT[0]

print(f"Service account is: {SERVICE_ACCOUNT}")

# Give permissions
! gcloud kms keys add-iam-policy-binding {KEY_ID} \
  --keyring={KEY_RING_ID} \
  --location={REGION} \
  --project={PROJECT_ID} \
  --member=serviceAccount:{SERVICE_ACCOUNT} \
  --role=roles/cloudkms.cryptoKeyEncrypterDecrypter

Service account is: service-759209241365@gcp-sa-aiplatform.iam.gserviceaccount.com
Updated IAM policy for key [your_cmek_key_id].
bindings:
- members:
  - serviceAccount:service-759209241365@gcp-sa-aiplatform.iam.gserviceaccount.com
  role: roles/cloudkms.cryptoKeyEncrypterDecrypter
etag: BwXaMQJNuYk=
version: 1


Create the full resource identifier for the created key

In [26]:
ENCRYPTION_SPEC_KEY_NAME = key_id.name

## Initialize Vertex SDK for Python

Initialize the *client* for Vertex AI

All resources created during this Notebook run will encrypted with the encryption key created above.

You can override the encryption key at each function call.

### Initialize Vertex AI SDK for Python

Initialize the Vertex AI SDK for Python for your project, bucket, and corresponding encryption key.

All resources created during this session are encrypted with the encryption key you created.

*Note:* You can override the encryption key at each function call.

In [28]:
aip.init(
    project=PROJECT_ID,
    staging_bucket=BUCKET_NAME,
    location=REGION,
    encryption_spec_key_name=ENCRYPTION_SPEC_KEY_NAME,
)

#### Location of Cloud Storage training data.

Now set the variable `IMPORT_FILE` to the location of the CSV index file in Cloud Storage.

In [29]:
IMPORT_FILE = 'gs://cloud-samples-data/vision/automl_classification/flowers/all_data_v2.csv'

# Create `Vertex AI ImageDataset` resource

Next, you create an `ImageDataset` resource, which will be encrypted using your encryption key.

In [40]:
dataset = aip.ImageDataset.create(
    display_name="flowers_" + TIMESTAMP,
    gcs_source=[IMPORT_FILE],
    import_schema_uri=aip.schema.dataset.ioformat.image.single_label_classification,
)

print(dataset.resource_name)

ResourceExhausted: 429 The following quota metrics exceed quota limits: aiplatform.googleapis.com/in_use_customer_managed_encryption_keys

# Launch a Training Job to Create a Model

Train an AutoML Image Classification model.

In [ ]:
job = aiplatform.AutoMLImageTrainingJob(
    display_name="flowers_" + TIMESTAMP,
    prediction_type="classification",
    multi_label=False,
    model_type="CLOUD",
    base_model=None,
)

# This will take around half an hour to run
model = job.run(
    dataset=ds,
    model_display_name="flowers_" + TIMESTAMP,
    training_fraction_split=0.6,
    validation_fraction_split=0.2,
    test_fraction_split=0.2,
    budget_milli_node_hours=8000,
    disable_early_stopping=False,
)

# Deploy Your Model

Deploy your model, then wait until the model FINISHES deployment before proceeding to prediction.

In [ ]:
endpoint = model.deploy()

# Predict on Endpoint
- Take one sample from the data imported to the dataset
- This sample will be encoded to base64 and passed to the endpoint for prediction

In [ ]:
test_item = !gsutil cat $IMPORT_FILE | head -n1
test_item, test_label = str(test_item[0]).split(",")

print(test_item, test_label)

In [ ]:
import base64

import tensorflow as tf

with tf.io.gfile.GFile(test_item, "rb") as f:
    content = f.read()

# The format of each instance should conform to the deployed model's prediction input schema.
instances_list = [{"content": base64.b64encode(content).decode("utf-8")}]

prediction = endpoint.predict(instances=instances_list)

print(prediction)

# Undeploy Model from Endpoint

In [ ]:
endpoint.undeploy_all()

In [ ]:
# missing
endpoint.delete()
model.delete()
dataset.delete()

! gcloud kms keys versions destroy key-version \
  --key key {KEY_ID} \
  --keyring={KEY_RING_ID} \
  --location={REGION} 

In [36]:
! gcloud kms keys list --location {REGION} --keyring {KEY_RING_ID}

NAME                                                                                                        PURPOSE          ALGORITHM                    PROTECTION_LEVEL  LABELS  PRIMARY_ID  PRIMARY_STATE
projects/andy-1234-221921/locations/us-central1/keyRings/your_cmek_key_ring_id/cryptoKeys/your_cmek_key_id  ENCRYPT_DECRYPT  GOOGLE_SYMMETRIC_ENCRYPTION  SOFTWARE                  1           ENABLED
